In [19]:
## 1. 程序初始化
import time
import random
import pickle
import logging
import argparse
import itertools
import numpy as np
from random import sample
import matplotlib.pyplot as plt
from collections import Counter
from scipy.spatial.distance import cosine

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical

import gym
from gym import spaces
from gym.utils import seeding

In [20]:
def stack(flat, layers=16):
    """Convert an [4, 4] representation into [4, 4, layers] with one layers for each value."""
    # representation is what each layer represents
    representation = 2 ** (np.arange(layers, dtype=int) + 1)

    # layered is the flat board repeated layers times
    layered = np.repeat(flat[:, :, np.newaxis], layers, axis=-1)

    # Now set the values in the board to 1 or zero depending whether they match representation.
    # Representation is broadcast across a number of axes
    layered = np.where(layered == representation, 1, 0)

    return layered

In [21]:
## 3. 创建环境
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = itertools.tee(iterable)
    next(b, None)
    return zip(a, b)

class IllegalMove(Exception):
    pass

In [22]:
# 游戏环境
class Game2048Env(gym.Env):
    metadata = {'render.modes': ['ansi', 'human', 'rgb_array']}

    def __init__(self):
        # Definitions for game. Board must be square.
        self.size = 4
        self.w = self.size
        self.h = self.size
        self.squares = self.size * self.size

        # Maintain own idea of game score, separate from rewards
        self.score = 0

        # Members for gym implementation
        self.action_space = spaces.Discrete(4)
        # Suppose that the maximum tile is as if you have powers of 2 across the board.
        layers = self.squares
        self.observation_space = spaces.Box(0, 1, (self.w, self.h, layers), int)
        self.set_illegal_move_reward(-20)
        self.set_max_tile(None)

        # Size of square for rendering
        self.grid_size = 70

        # Initialise seed
        self.seed()

        # Reset ready for a game
        self.reset()

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def set_illegal_move_reward(self, reward):
        """Define the reward/penalty for performing an illegal move. Also need
            to update the reward range for this."""
        # Guess that the maximum reward is also 2**squares though you'll probably never get that.
        # (assume that illegal move reward is the lowest value that can be returned
        self.illegal_move_reward = reward
        self.reward_range = (self.illegal_move_reward, float(2 ** self.squares))

    def set_max_tile(self, max_tile):
        """Define the maximum tile that will end the game (e.g. 2048). None means no limit.
           This does not affect the state returned."""
        assert max_tile is None or isinstance(max_tile, int)
        self.max_tile = max_tile

    # Implement gym interface
    def step(self, action):
        """Perform one step of the game. This involves moving and adding a new tile."""
        logging.debug("Action {}".format(action))
        score = 0
        done = None
        info = {
            'illegal_move': False,
        }
        try:
            score = float(self.move(action))
            if score > 0:
                score = score
                # score = math.log2(score)
            if score < 0:
                score = 0
            self.score += score
            assert score <= 2 ** (self.w * self.h)
            self.add_tile()
            done = self.isend()
            reward = float(score)

        except IllegalMove:
            logging.debug("Illegal move")
            info['illegal_move'] = True
            done = True
            reward = self.illegal_move_reward
            # reward=0

        # print("Am I done? {}".format(done))
        info['highest'] = self.highest()

        # Return observation (board state), reward, done and info dict
        return stack(self.Matrix), reward, done, info
    def reset(self):
        self.Matrix = np.zeros((self.h, self.w), int)
        self.score = 0

        logging.debug("Adding tiles")
        self.add_tile()
        self.add_tile()

        return stack(self.Matrix)




    # Implement 2048 game
    def add_tile(self):
        """Add a tile, probably a 2 but maybe a 4"""
        possible_tiles = np.array([2, 4])
        tile_probabilities = np.array([0.9, 0.1])
        val = self.np_random.choice(possible_tiles, 1, p=tile_probabilities)[0]
        empties = self.empties()
        assert empties.shape[0]
        empty_idx = self.np_random.choice(empties.shape[0])
        empty = empties[empty_idx]
        logging.debug("Adding %s at %s", val, (empty[0], empty[1]))
        self.set(empty[0], empty[1], val)

    def get(self, x, y):
        """Return the value of one square."""
        return self.Matrix[x, y]

    def set(self, x, y, val):
        """Set the value of one square."""
        self.Matrix[x, y] = val

    def empties(self):
        """Return a 2d numpy array with the location of empty squares."""
        return np.argwhere(self.Matrix == 0)

    def highest(self):
        """Report the highest tile on the board."""
        return np.max(self.Matrix)

    def board_total(self):
        """Calculate the total value of all tiles on the board."""
        return np.sum(self.Matrix)


    def move(self, direction, trial=False):
        """Perform one move of the game. Shift things to one side then,
        combine. directions 0, 1, 2, 3 are up, right, down, left.
        Returns the maximum score that [would have] got from the move."""
        if not trial:
            if direction == 0:
                logging.debug("Up")
            elif direction == 1:
                logging.debug("Right")
            elif direction == 2:
                logging.debug("Down")
            elif direction == 3:
                logging.debug("Left")

        changed = False
        scores = []  # 修改为列表，用于存储每次移动得到的分数
        dir_div_two = int(direction / 2)
        dir_mod_two = int(direction % 2)
        shift_direction = dir_mod_two ^ dir_div_two  # 0 for towards up left, 1 for towards bottom right

        # Construct a range for extracting row/column into a list
        rx = list(range(self.w))
        ry = list(range(self.h))

        if dir_mod_two == 0:
            # Up or down, split into columns
            for y in range(self.h):
                old = [self.get(x, y) for x in rx]
                (new, ms) = self.shift(old, shift_direction)
                scores.append(ms)  # 添加到分数列表中
                if old != new:
                    changed = True
                    if not trial:
                        for x in rx:
                            self.set(x, y, new[x])
        else:
            # Left or right, split into rows
            for x in range(self.w):
                old = [self.get(x, y) for y in ry]
                (new, ms) = self.shift(old, shift_direction)
                scores.append(ms)  # 添加到分数列表中
                if old != new:
                    changed = True
                    if not trial:
                        for y in ry:
                            self.set(x, y, new[y])
        if not changed:
            raise IllegalMove
            # 打印分数列表和最大分数

        # 获取列表中的最大值作为 move_scores 返回
        move_scores = max(scores) if scores else 0
        # print("Scores from this move:", scores)
        # print("Maximum score from this move:", move_scores)
        return move_scores

    def combine(self, shifted_row):
        """Combine same tiles when moving to one side. This function always
           shifts towards the left. Also count the score of combined tiles."""
        move_score = 0
        combined_row = [0] * self.size
        skip = False
        output_index = 0
        for p in pairwise(shifted_row):
            if skip:
                skip = False
                continue
            combined_row[output_index] = p[0]
            if p[0] == p[1]:
                combined_row[output_index] += p[1]
                move_score += p[0] + p[1]
                # Skip the next thing in the list.
                skip = True
            output_index += 1
        if shifted_row and not skip:
            combined_row[output_index] = shifted_row[-1]

        return (combined_row, move_score)

    def shift(self, row, direction):
        """Shift one row left (direction == 0) or right (direction == 1), combining if required."""
        length = len(row)
        assert length == self.size
        assert direction == 0 or direction == 1

        # Shift all non-zero digits up
        shifted_row = [i for i in row if i != 0]

        # Reverse list to handle shifting to the right
        if direction:
            shifted_row.reverse()

        (combined_row, move_score) = self.combine(shifted_row)

        # Reverse list to handle shifting to the right
        if direction:
            combined_row.reverse()

        assert len(combined_row) == self.size
        return (combined_row, move_score)

    def isend(self):
        """Has the game ended. Game ends if there is a tile equal to the limit
           or there are no legal moves. If there are empty spaces then there
           must be legal moves."""

        # if self.max_tile is not None and self.highest() == self.max_tile:
        #     return True

        for direction in range(4):
            try:
                self.move(direction, trial=True)
                # Not the end if we can do any move
                return False
            except IllegalMove:
                pass
        return True

    def get_board(self):
        """Retrieve the whole board, useful for testing."""
        return self.Matrix

    def set_board(self, new_board):
        """Retrieve the whole board, useful for testing."""
        self.Matrix = new_board

In [23]:
env = Game2048Env()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
# 测试模型函数:
def infer_model(model, env, threshold=1024, num_trials=100):
    random_action_count = 0
    model_action_count = 0
    success_count = 0

    for _ in range(num_trials):
        s = env.reset()
        last_s = None
        actions_taken = []  # 初始化已尝试的动作列表

        while True:
            s_tensor = torch.FloatTensor(np.expand_dims(s, axis=0)).to(device)
            if np.array_equal(last_s, s):
                # 去除已尝试的动作
                available_actions = [i for i in range(4) if i not in actions_taken]

                # 如果没有剩余动作尝试，结束当前尝试
                if not available_actions:
                    break

                # 选择一个新的动作
                a = random.choice(available_actions)
                actions_taken.append(a)  # 记录尝试过的动作
                random_action_count += 1
            else:
                with torch.no_grad():
                    logits = model(s_tensor)
                    policy = F.softmax(logits, dim=1)
                    m = Categorical(policy)
                    a = m.sample().item()
                model_action_count += 1
                # 重置动作列表
                actions_taken = []

            last_s = np.array(s)
            s_, r, done, info = env.step(a)
            if done:
                if env.highest() >= threshold:
                    success_count += 1
                break
            s = s_
    return success_count, random_action_count, model_action_count


In [25]:
# 加载模型和相关数据
with open("Test.pkl", "rb") as file:
    model_data = pickle.load(file)

# 提取模型
loaded_model = model_data['model']
# 如果模型保存时未被转移到 CPU，需确保其设备与当前环境一致
loaded_model = loaded_model.to('cuda' if torch.cuda.is_available() else 'cpu')

# 如果需要提取其他保存的信息，可以参考以下：
standard_score = model_data['other_data']['standard_score']
success_rate = model_data['other_data']['success_rate']
rounds = model_data['other_data']['rounds']

AttributeError: Can't get attribute 'NetLowReward' on <module '__main__'>

In [ ]:
# 在 1000 局中测试模型的性能
threshold_1024 = 1024
threshold_2048 = 2048

# 统计胜率
success_count_1024, random_action_count_1024, model_action_count_1024 = infer_model(
    loaded_model, env, threshold=threshold_1024, num_trials=1000
)

success_count_2048, random_action_count_2048, model_action_count_2048 = infer_model(
    loaded_model, env, threshold=threshold_2048, num_trials=1000
)

# 打印结果
print(f"在 1000 局中达到 {threshold_1024} 的胜率: {success_count_1024 / 1000:.2%}")
print(f"在 1000 局中达到 {threshold_2048} 的胜率: {success_count_2048 / 1000:.2%}")
